<a href="https://colab.research.google.com/github/Peridan9/PINN-module-for-aircraft-simulator/blob/Daniella/PinnProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightning pytorch_lightning
!pip install -U neptune

In [2]:
import torch
import lightning as L
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import BatchSizeFinder, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger
import neptune
import os

### Loading and PreProcessing the Data


txt to csv if needed

In [3]:
# import csv

# name_open = 'Data.txt'
# name_new = 'datatest2.csv'
# delimiter = '|'  # Adjust this if your actual delimiter is different

# # Read from text file and write to CSV file
# with open(name_open, 'r') as f:
#     # Read the lines
#     lines = f.readlines()

# # Extract headers from the first line
# headers = [header.strip() for header in lines[0].split(delimiter)]

# # Open a new CSV file to write the data
# with open(name_new, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile, escapechar='\\')  # Specify escapechar to handle special characters

#     # Write headers
#     writer.writerow(headers)

#     # Write rows
#     for line in lines[1:]:
#         row = [item.strip() for item in line.split(delimiter)]
#         writer.writerow(row)

# print(f"Conversion completed. Data written to {name_new}")


In [4]:
#relevent columes

relevent_features = ['_totl,_time','Vtrue,_ktgs','pitch,__deg','_roll,__deg','hding,_true','__lat,__deg','__lon,__deg','___CG,ftMSL','____X,____m','____Y,____m','____Z,____m','___vX,__m/s','___vY,__m/s','___vZ,__m/s'
                     ,'_elev,stick','ailrn,stick','ruddr,stick','thro1,engin'
]

train_data = pd.read_csv('data/Data2.csv', skiprows=1)
train_data = train_data.dropna(axis=1, how='all')
train_data = train_data[relevent_features]
train_data

val_data = pd.read_csv('data/Data3.csv', skiprows=1)
val_data = val_data.dropna(axis=1, how='all')
val_data = val_data[relevent_features]
val_data

test_data = pd.read_csv('data/Data4.csv', skiprows=1)
test_data = test_data.dropna(axis=1, how='all')
test_data = test_data[relevent_features]
test_data

,"_totl,_time","Vtrue,_ktgs","pitch,__deg","_roll,__deg","hding,_true","__lat,__deg","__lon,__deg","___CG,ftMSL","____X,____m","____Y,____m","____Z,____m","___vX,__m/s","___vY,__m/s","___vZ,__m/s","_elev,stick","ailrn,stick","ruddr,stick","thro1,engin"
0,0.05025,0.03180,2.45996,-0.01914,299.11221,45.58352,-122.56671,20.68024,33814.53125,-90.58956,-9374.13574,-0.00504,0.01675,0.00647,0.00000,0.00000,0.00000,0.0
1,0.14867,0.05056,2.57164,-0.09045,299.10840,45.58352,-122.56671,20.68480,33814.53125,-90.58817,-9374.13379,0.00650,0.00881,0.02463,0.00000,0.00000,0.00000,0.0
2,0.19892,0.05729,2.64648,-0.13279,299.10690,45.58352,-122.56671,20.68548,33814.53125,-90.58796,-9374.13281,0.01017,0.00144,0.02851,0.00000,0.00000,0.00000,0.0
3,0.29942,0.06552,2.81113,-0.21332,299.10550,45.58352,-122.56671,20.68469,33814.53125,-90.58820,-9374.12988,0.01499,-0.00407,0.03018,0.00000,0.00000,0.00000,0.0
4,0.34967,0.06621,2.89730,-0.24817,299.10544,45.58352,-122.56671,20.68395,33814.53516,-90.58843,-9374.12793,0.01667,-0.00479,0.02932,0.00000,0.00000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,749.95093,120.01694,4.55561,15.65379,18.49638,45.74081,-122.58516,3136.71533,32289.12891,817.60822,-26852.25195,19.50777,4.89486,-58.59840,-0.00240,-0.05574,0.00640,1.0
8369,750.00122,119.93780,4.56340,15.52673,18.62847,45.74084,-122.58514,3137.57959,32290.11133,817.85431,-26855.19531,19.62884,4.89843,-58.51509,-0.00229,-0.05538,0.00642,1.0
8370,750.05145,119.85892,4.57110,15.40041,18.75956,45.74087,-122.58514,3138.44434,32291.10156,818.10052,-26858.13281,19.74856,4.90193,-58.43207,-0.00218,-0.05508,0.00640,1.0
8371,750.10168,119.78024,4.57869,15.27468,18.88974,45.74089,-122.58512,3139.30981,32292.09766,818.34698,-26861.06641,19.86696,4.90534,-58.34925,-0.00208,-0.05483,0.00635,1.0


# CREATING A MODEL

In [5]:
class FlightDataset(Dataset):
    def __init__(self, df, seq_len, scaler=None):
        # Keeping only the relevant features
        relevant_features = ['_totl,_time', 'Vtrue,_ktgs', 'pitch,__deg', '_roll,__deg', 'hding,_true', '__lat,__deg', '__lon,__deg', '___CG,ftMSL', '____X,____m', '____Y,____m', '____Z,____m', '___vX,__m/s', '___vY,__m/s', '___vZ,__m/s', '_elev,stick', 'ailrn,stick', 'ruddr,stick', 'thro1,engin']

        self.df = df[relevant_features].copy()  # Use copy to avoid modifying the original DataFrame

        # Changing the names of the features for easier use
        self.df.columns = ['time', 'vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'elev_stick', 'ailrn_stick', 'ruddr_stick', 'throttle']

        if scaler is None:
            self.scaler = StandardScaler()
            self.df = pd.DataFrame(self.scaler.fit_transform(self.df), columns=self.df.columns)
        else:
            self.scaler = scaler
            self.df = pd.DataFrame(self.scaler.transform(self.df), columns=self.df.columns)

        # Define sequence length
        self.seq_len = seq_len

    def __len__(self):
        # Making the length shorter by seq_len so we always have a next_state
        return len(self.df) - 1

    def __getitem__(self, idx):
        if idx < self.seq_len:
            padding = np.zeros((self.seq_len - idx, len(self.df.columns)))
            current_state = np.vstack((padding, self.df.iloc[:idx].values.astype(float)))
        else:
            current_state = self.df.iloc[idx - self.seq_len:idx].values.astype(float)

        # Next state is the state at idx + seq_len
        next_state = self.df.iloc[idx + 1][['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']].values.astype(float)

        return torch.tensor(current_state, dtype=torch.float32), torch.tensor(next_state, dtype=torch.float32)

        # if idx < self.seq_len:
        #     # Pad the sequence with zeros if idx is less than seq_len
        #     padding = np.zeros((self.seq_len - idx, len(self.df.columns)))
        #     current_state = np.vstack((padding, self.df.iloc[:idx + 1].values.astype(float)))
        # else:
        #     # Regular sequence extraction
        #     current_state = self.df.iloc[idx - self.seq_len + 1:idx + 1].values.astype(float)

        # # Next state is the state at idx + 1
        # next_state = self.df.iloc[idx + 1][['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']].values.astype(float)

        # return torch.tensor(current_state, dtype=torch.float32), torch.tensor(next_state, dtype=torch.float32)

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)


In [6]:
class FlightLSTM(pl.LightningModule):
    def __init__(self,input_dim,hidden_dim,num_layers,output_dim,lr=0.001):
        super(FlightLSTM,self).__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.lr = lr
        self.criterion = nn.MSELoss(reduction='none')

        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

    def forward(self,x):
        out,_ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

    def compute_metrics(self, y_true, y_pred):
        mse = np.mean((y_true - y_pred) ** 2, axis=0)
        mae = np.mean(np.abs(y_true - y_pred), axis=0)

        return mse, mae

    def training_step(self,batch,batch_idx):
        x, y = batch
        output = self(x)
        loss = self.criterion(output, y).mean()
        self.log("train/batch/loss", loss, prog_bar=False)

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"train/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"train/batch/mae_{feature}", mae[i], prog_bar=False)

        self.training_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}

    def on_train_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.training_step_outputs])
        mse = np.vstack([output["mse"] for output in self.training_step_outputs])
        mae = np.vstack([output["mae"] for output in self.training_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.training_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.training_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("train/epoch/loss", loss.mean())  # Log training epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"train/epoch/mse_{feature}", mse_all[i])  # Log training epoch MSE for each feature
            self.log(f"train/epoch/mae_{feature}", mae_all[i])  # Log training epoch MAE for each feature
        self.log("train/epoch/r2", r2_all)  # Log training epoch R²


        print("Training Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = self.criterion(output, y).mean()

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"val/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"val/batch/mae_{feature}", mae[i], prog_bar=False)

        self.validation_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}

    def on_validation_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.validation_step_outputs])
        mse = np.vstack([output["mse"] for output in self.validation_step_outputs])
        mae = np.vstack([output["mae"] for output in self.validation_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.validation_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.validation_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("val/epoch/loss", loss.mean())  # Log validation epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"val/epoch/mse_{feature}", mse_all[i])  # Log validation epoch MSE for each feature
            self.log(f"val/epoch/mae_{feature}", mae_all[i])  # Log validation epoch MAE for each feature
        self.log("val/epoch/r2", r2_all)  # Log validation epoch R²

        print("Validation Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.validation_step_outputs.clear()  # free memory

    def test_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = self.criterion(output, y).mean()

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"test/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"test/batch/mae_{feature}", mae[i], prog_bar=False)

        self.test_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}

    def on_test_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.test_step_outputs])
        mse = np.vstack([output["mse"] for output in self.test_step_outputs])
        mae = np.vstack([output["mae"] for output in self.test_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.test_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.test_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("test/epoch/loss", loss.mean())  # Log test epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"test/epoch/mse_{feature}", mse_all[i])  # Log test epoch MSE for each feature
            self.log(f"test/epoch/mae_{feature}", mae_all[i])  # Log test epoch MAE for each feature
        self.log("test/epoch/r2", r2_all)  # Log test epoch R²

        print("Test Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.test_step_outputs.clear()  # free memory

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer


In [7]:
def predictions_stats(self, test_data):
    predictions = []
    actual_data = []

    for batch in test_data:
        x, y = batch
        output = self.forward(x)
        predictions.append(output.cpu().detach().numpy())
        actual_data.append(y.cpu().detach().numpy())

    predictions = np.vstack(predictions)
    actual_data = np.vstack(actual_data)

    columns = ['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']
    for i, col in enumerate(columns):
      calculate_anova_table(col, actual_data[i], predictions[i])

def calculate_anova_table(feature, y_trues, y_preds):
    # Anova Table Calculations
    n = len(y_trues)
    y_mean = np.mean(y_trues)

    SSR = np.sum((y_preds[i] - y_mean)**2 for i in range(n))
    SSE = np.sum((y_trues[i] - y_preds[i])**2 for i in range(n))
    SSTO = np.sum((y_trues[i] - y_mean)**2 for i in range(n))

    # Degrees of Freedom
    df_regression = 1
    df_residual = n - 2
    df_total = n - 1

    # Mean Squares
    MSR = SSR / df_regression
    MSE = SSE / df_residual

    # F-statistic
    F = MSR / MSE

    # You can print or store these values as needed
    print(f"\nANOVA Table for {feature}:")
    print("Source of Variation\t | DF \t\t | SS")
    print(f"SSR\t\t\t | {df_regression} \t\t | {SSR:.2f}")
    print(f"SSE\t\t\t | {df_residual}   \t\t | {SSE:.2f}")
    print(f"SSTO\t\t\t | {df_total}     \t | {SSTO:.2f}")
    print(f"F-statistic: {F:.2f}")

In [8]:
def seed_everything(seed=42):
    # random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
seed_everything()

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [9]:
sequance_length = 50
train_dataset = FlightDataset(train_data, sequance_length)
fitted_scaler = train_dataset.scaler
val_dataset = FlightDataset(val_data, sequance_length, fitted_scaler)
test_dataset = FlightDataset(test_data, sequance_length, fitted_scaler)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
# # Assuming train_loader is your dataloader
# for batch in train_loader:
#     # Unpack the batch (assuming it contains inputs and targets)
#     inputs, targets = batch

#     # Print the inputs and targets to understand their structure
#     print("Inputs: ", inputs)
#     print("Targets: ", targets)

#     # Print the size of the inputs
#     print("Size of inputs: ", inputs.size())

#     # Break the loop after the first batch
#     break


In [11]:
neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4ZGJmNTQyNy04OWFhLTQ0YzMtYTA5Ni1hNTc1NjJmMGYxMjkifQ==",
    project="kapustya/PIML-TEST",
    tags=["notebook"],
    log_model_checkpoints=False
)

In [12]:
seed_everything()
input_dim = len(train_data.columns)
hidden_dim = 128
num_layers = 2
output_dim = 9
model = FlightLSTM(input_dim, hidden_dim, num_layers, output_dim)
trainer = pl.Trainer(
    logger = neptune_logger,
    max_epochs=10
    )
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)
# predictions_stats(model, test_loader)
# neptune_logger.experiment.stop()


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kapustya/PIML-TEST/e/PIM1-4


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | lstm      | LSTM    | 207 K  | train
1 | fc        | Linear  | 1.2 K  | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
209 K     Trainable params
0         Non-trainable params
209 K     Total params
0.836     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Validation Epoch End: Loss: 21.203953 MSE: [1.1085367e+02 1.8449553e+01 5.7121838e+01 1.5304285e-01 2.8710075e-02
 2.2179408e+00 4.2854555e-02 4.0184795e-03 1.9639244e+00] MAE: [10.528671    4.2952423   7.5578938   0.38952175  0.16783875  1.4891117
  0.20581418  0.05104988  1.4007993 ] R²: -165245522010.11557


...

Validation Epoch End: Loss: 6.198344 MSE: [38.829567    3.9999506   8.43708     0.98933613  0.7484696   0.47113323
  0.5379105   0.57147217  1.2001739 ] MAE: [5.866803   1.6231959  2.2161112  0.8358344  0.5958862  0.5828489
 0.6722978  0.5508934  0.90900725] R²: -172.4455787204089
Training Epoch End: Loss: 0.14745411 MSE: [0.09390297 0.07799783 0.10081865 0.11734521 0.31167734 0.10899414
 0.12368408 0.11528019 0.27738672] MAE: [0.14176534 0.17642412 0.18807417 0.22857375 0.27215987 0.17054455
 0.22314252 0.19689561 0.28753716] R²: 0.8520452241491294


Testing: |          | 0/? [00:00<?, ?it/s]

[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 1229.0
Test Epoch End: Loss: 4.0766287 MSE: [20.77572     3.372943    6.7016177   0.88623524  0.4227288   0.8071281
  1.4075855   0.53328836  1.7824072 ] MAE: [4.03552    1.3904998  1.8830914  0.74821556 0.27297786 0.83689654
 1.0569122  0.5408521  1.1819124 ] R²: -10.383380393153129


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/batch/mae_alt     │    1.8847393989562988     │
│   test/batch/mae_hding    │    0.8366779685020447     │
│    test/batch/mae_lat     │    4.0381245613098145     │
│    test/batch/mae_lon     │    1.3921078443527222     │
│   test/batch/mae_pitch    │    0.7487176060676575     │
│    test/batch/mae_roll    │    0.27220505475997925    │
│     test/batch/mae_vx     │    1.0567384958267212     │
│     test/batch/mae_vy     │    0.5409992933273315     │
│     test/batch/mae_vz     │    1.1797523498535156     │
│    test/batch/mse_alt     │     6.710452079772949     │
│   test/batch/mse_hding    │    0.8068814277648926     │
│    test/batch/mse_lat     │    20.798444747924805     │
│    test/batch/mse_lon     │    3.3776743412017822     │
│   test/batch/mse_pitch    │    0.8872774243354797     │
│    test/batch/mse_roll    │    0.42237165570259094    │
│     test/batch/mse_vx     │    1.4075878858566284     │
│     test/batch/mse_vy     │    0.5337762236595154     │
│     test/batch/mse_vz     │    1.7745920419692993     │
│      test/epoch/loss      │     4.076628684997559     │
│    test/epoch/mae_alt     │    1.8830914497375488     │
│   test/epoch/mae_hding    │     0.836896538734436     │
│    test/epoch/mae_lat     │     4.035520076751709     │
│    test/epoch/mae_lon     │    1.3904998302459717     │
│   test/epoch/mae_pitch    │    0.7482155561447144     │
│    test/epoch/mae_roll    │    0.2729778587818146     │
│     test/epoch/mae_vx     │    1.0569121837615967     │
│     test/epoch/mae_vy     │    0.5408521294593811     │
│     test/epoch/mae_vz     │    1.1819124221801758     │
│    test/epoch/mse_alt     │     6.70161771774292      │
│   test/epoch/mse_hding    │    0.8071280717849731     │
│    test/epoch/mse_lat     │    20.775720596313477     │
│    test/epoch/mse_lon     │    3.3729429244995117     │
│   test/epoch/mse_pitch    │     0.886235237121582     │
│    test/epoch/mse_roll    │    0.4227288067340851     │
│     test/epoch/mse_vx     │    1.4075855016708374     │
│     test/epoch/mse_vy     │    0.5332883596420288     │
│     test/epoch/mse_vz     │    1.7824071645736694     │
│       test/epoch/r2       │    -10.383379936218262    │
└───────────────────────────┴───────────────────────────┘

[{'test/batch/mse_lat': 20.798444747924805,
  'test/batch/mae_lat': 4.0381245613098145,
  'test/batch/mse_lon': 3.3776743412017822,
  'test/batch/mae_lon': 1.3921078443527222,
  'test/batch/mse_alt': 6.710452079772949,
  'test/batch/mae_alt': 1.8847393989562988,
  'test/batch/mse_pitch': 0.8872774243354797,
  'test/batch/mae_pitch': 0.7487176060676575,
  'test/batch/mse_roll': 0.42237165570259094,
  'test/batch/mae_roll': 0.27220505475997925,
  'test/batch/mse_hding': 0.8068814277648926,
  'test/batch/mae_hding': 0.8366779685020447,
  'test/batch/mse_vx': 1.4075878858566284,
  'test/batch/mae_vx': 1.0567384958267212,
  'test/batch/mse_vy': 0.5337762236595154,
  'test/batch/mae_vy': 0.5409992933273315,
  'test/batch/mse_vz': 1.7745920419692993,
  'test/batch/mae_vz': 1.1797523498535156,
  'test/epoch/loss': 4.076628684997559,
  'test/epoch/mse_lat': 20.775720596313477,
  'test/epoch/mae_lat': 4.035520076751709,
  'test/epoch/mse_lon': 3.3729429244995117,
  'test/epoch/mae_lon': 1.390499

In [16]:
predictions_stats(model, test_loader)
neptune_logger.experiment.stop()


ANOVA Table for lat:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 15.70
SSE			 | 7   		 | 191.17
SSTO			 | 8     	 | 174.29
F-statistic: 0.57

ANOVA Table for lon:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 19.29
SSE			 | 7   		 | 99.44
SSTO			 | 8     	 | 174.31
F-statistic: 1.36

ANOVA Table for alt:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 58.64
SSE			 | 7   		 | 35.04
SSTO			 | 8     	 | 174.34
F-statistic: 11.71

ANOVA Table for pitch:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 87.76
SSE			 | 7   		 | 17.51
SSTO			 | 8     	 | 174.37
F-statistic: 35.09

ANOVA Table for roll:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 97.47
SSE			 | 7   		 | 13.74
SSTO			 | 8     	 | 174.39
F-statistic: 49.65

ANOVA Table for hding:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 99.70
SSE			 | 7   		 | 13.05
SSTO			 | 8     	 | 174.41
F-statistic: 53.48

ANOVA Table for vx:
Source of Variation	 | DF 		 | SS
SSR			 | 1 		 | 99.99
SSE			 | 7   		 | 13.01
SSTO			

<ipython-input-7-87d894adf8b1>:23: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  SSR = np.sum((y_preds[i] - y_mean)**2 for i in range(n))
<ipython-input-7-87d894adf8b1>:24: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  SSE = np.sum((y_trues[i] - y_preds[i])**2 for i in range(n))
<ipython-input-7-87d894adf8b1>:25: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  SSTO = np.sum((y_trues[i] - y_mean)**2 for i in range(n))
